---
title: "Getting Started with New Relic and Splunk - A Complete Guide from Installation to Microservices Integration"
author: "Vahram Poghosyan"
date: "2024-11-07"
categories: ["Software Engineering", "DevOps", "Observability and Monitoring", "NewRelic"]
format:
  html:
    code-fold: false
    code-line-numbers: false
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

# Introduction to NewRelic and Splunk

## NewRelic vs Splunk

NewRelic and Splunk are both powerful tools for monitoring and observability, but they have some key differences:

- **NewRelic** is more focused on **application performance monitoring** (**APM**), for which it offers a wide range of features, including **distributed tracing** (tracing the event flow across microservices).
- **Splunk** is a more focused on log aggregation and analysis.

Having said that, there's increasingly significant overlap between them as these products grow more and gain a diverse set of features. As of the time of writing this, each has its core competencies, and the two complement (rather than compete with) each other.

### NewRelic

#### How Does a New Relic Agent Collect Metrics from My Application?

A New Relic **agent** (which comes in many different flavors, depending on your choice of a backend language) collects metrics by either *instrumenting* your application's code or attaching to its runtime environment (e.g. on the same JVM that your Java (or Scala) application is running on). Here's how it works:

1. **Instrumentation**: The agent integrates with your application at the **code** level or **runtime environment** level
   - Code level instrumentation essentially means that the NewRelic agent is imported into the application's source code itself, giving the developer far more visibility and control over the application's logs
   - Code level instrumentation is often called **monkey patching** and it's typically used for dynamic languages like Python or JS (which are interpreted, not compiled)
   - However, instrumenting our code is not always practical, so NewRelic offers a more cookie-cutter solution that works right out of the box -- **runtime environment** instrumentation. This means that the agent is started with the application itself, and attaches to the running process
   - The runtime environment level of NewRelic integration is sometimes also called **bytecode manipulation**, and it's usually the approach taken when dealing with languages like Java or Scala (statically typed, compiled languages). It's a method that inserts monitoring **hooks** into your application which collect real-time data on its health and performance
  
2. **Data Collection**: The NewRelic agent's hooks collect data on various aspects of your application's performance. They either do this automatically, by detecting the frameworks used in the application and *wrapping* key methods and functions that handle an incoming request within a framework (e.g. an API endpoint's logic implemented in an HTTP library of the given language) within the bytecode itself (in the runtime environment), or by the developer's manual addition of **instrumentation points** (which are the places in the source code where the agent will *insert* its hooks). The data collected includes **metrics**, **events**, **logs**, and **traces** (also known as **MELT**), representing the four key datapoints used in **Observability** to comprehensively monitor and understand the real-time health and performance of your microservices 
   - **Metrics** such as **response times**, number and duration of **database queries**, number and duration of **external service calls**, the **memory and CPU usage** of the application (which, actually, come from container-level logs -- i.e. Docker or Kubernetes -- than the runtime of the application itself), and **error rates**.

3. **Data Transmission**: The collected data is asynchronously sent to New Relic's cloud-based platform through secure communication channels. This will be explored in more detail in a separate paragraph (#TODO).

4. **Analysis and Visualization**: Once the data reaches New Relic, it is processed and made available in NewRelic. New Relic stores the data in its **New Relic Telemetry Database** (**NRDB**) across multiple clouds in the United States and the European Union. New Relic uses AWS and Azure for data storage. This data can then be used to generate **dashboards**, **alerts**, and **reports**, providing real-time insights into your application's health and performance.

#### How Do You Install a New Relic Agent?

The installation process varies depending on the programming language and environment of your application. Below is a general step-by-step guide:

1. **Sign Up and Obtain License Key**:

   - Create an account on [New Relic's website](https://newrelic.com/).
   - Obtain your unique license key from the New Relic dashboard.

2. **Choose the Right Agent**:

   New Relic provides agents for various languages and platforms:

   | Language/Platform | Agent                                                   |
   |-------------------|---------------------------------------------------------|
   | **Java**          | `newrelic.jar`                                          |
   | **.NET**          | New Relic .NET Agent                                    |
   | **Node.js**       | `newrelic` npm package                                  |
   | **Python**        | `newrelic` pip package                                  |
   | **Ruby**          | `newrelic_rpm` gem                                      |
   | **PHP**           | New Relic PHP Agent                                     |
   | **Go**            | New Relic Go Agent                                      |

3. **Install the Agent**:

   - **For Java**:
     - Download the `newrelic.jar` file.
     - Add the agent to your JVM startup parameters:

       ```shell
       -javaagent:/path/to/newrelic.jar
       ```

   - **For Node.js**:
     - Install via npm:

       ```shell
       npm install newrelic
       ```

     - Require New Relic at the top of your main JavaScript file:

       ```javascript
       require('newrelic');
       ```

   - **For Python**:
     - Install via pip:

       ```shell
       pip install newrelic
       ```

     - Initialize the agent in your application's entry point.

4. **Configure the Agent**:

   - Update the configuration file (e.g., `newrelic.yml` for Java or `newrelic.js` for Node.js) with your license key and application name.
   - Set any additional settings like logging level or transaction tracing as needed.

5. **Restart Your Application**:

   - The agent begins collecting data when the application restarts with the agent enabled.

6. **Verify Installation**:

   - Log in to your New Relic dashboard to confirm that data is being reported.

#### Where Does It Sit in a Microservices Application?

In a microservices architecture, each service operates independently, often across different environments or platforms. Here's how the New Relic agent fits in:

1. **Agent in Each Microservice**:

   - Install and configure the New Relic agent within each microservice you want to monitor.
   - This ensures that performance data is collected at the granular level of each service.

2. **Integration Points**:

   - **Within the Application Code**: For languages like Node.js or Python, the agent is imported directly into the application code.
   - **At the Runtime Level**: For compiled languages like Java or .NET, the agent attaches to the runtime environment via startup parameters.

3. **Containerized Environments**:

   - If your microservices run in containers (e.g., Docker, Kubernetes), include the agent installation and configuration in your container images.
   - Alternatively, use sidecar containers for the agents.

4. **Distributed Tracing**:

   - New Relic supports distributed tracing, allowing you to trace requests as they flow through different microservices.
   - Ensure that distributed tracing is enabled in the agent configuration to correlate data across services.

5. **Data Aggregation**:

   - All agents report back to New Relic's centralized platform, where data is aggregated.
   - You can view the performance of individual microservices or the entire application stack.
  
#### Uuerying NewRelic: Query Language (NRQL)

### Splunk

#### Querying Splunk